## Setup

In [10]:
import tensorflow as tf
tf.enable_eager_execution()
import numpy as np

## Pipe the dataset to a model

In [11]:
X_train = np.load("X_train.npy")
y_train = np.load("y_train.npy")
X_train_preprocessed = X_train / 128
X_train_preprocessed -= 1
print("X_train_preprocessed:", X_train_preprocessed)
print("X_train_preprocessed.shape:", X_train_preprocessed.shape)
print("y_train:", y_train)
print("y_train.shape:", y_train.shape)

X_train_preprocessed: [[[[ 0.2421875  0.2109375  0.21875  ]
   [ 0.2421875  0.2109375  0.21875  ]
   [ 0.2421875  0.2109375  0.21875  ]
   ...
   [ 0.2578125  0.1875     0.2109375]
   [ 0.234375   0.203125   0.1953125]
   [ 0.21875    0.2109375  0.1953125]]

  [[ 0.2421875  0.2109375  0.21875  ]
   [ 0.2421875  0.2109375  0.21875  ]
   [ 0.2421875  0.2109375  0.21875  ]
   ...
   [ 0.2578125  0.1875     0.2109375]
   [ 0.234375   0.203125   0.1953125]
   [ 0.21875    0.2109375  0.1953125]]

  [[ 0.2421875  0.2109375  0.21875  ]
   [ 0.2421875  0.2109375  0.21875  ]
   [ 0.25       0.203125   0.21875  ]
   ...
   [ 0.2578125  0.1875     0.1953125]
   [ 0.234375   0.203125   0.1953125]
   [ 0.21875    0.2109375  0.1953125]]

  ...

  [[-0.625     -0.6171875 -0.5859375]
   [-0.65625   -0.6484375 -0.6171875]
   [-0.6640625 -0.65625   -0.6171875]
   ...
   [ 0.4609375  0.4140625  0.4296875]
   [ 0.4765625  0.40625    0.4296875]
   [ 0.46875    0.3984375  0.40625  ]]

  [[-0.625     -0.63281

In [12]:
label_names = ['undamaged', 'damaged']

In [13]:
inception_v3 = tf.keras.applications.InceptionV3(input_shape=(224, 224, 3), include_top=False)
# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(inception_v3.layers):
    print(i, layer.name)
# we will freeze the first 249 layers and unfreeze the rest:
for layer in inception_v3.layers[:249]:
    layer.trainable = False
for layer in inception_v3.layers[249:]:
    layer.trainable = True

0 input_5
1 conv2d_376
2 batch_normalization_376
3 activation_376
4 conv2d_377
5 batch_normalization_377
6 activation_377
7 conv2d_378
8 batch_normalization_378
9 activation_378
10 max_pooling2d_16
11 conv2d_379
12 batch_normalization_379
13 activation_379
14 conv2d_380
15 batch_normalization_380
16 activation_380
17 max_pooling2d_17
18 conv2d_384
19 batch_normalization_384
20 activation_384
21 conv2d_382
22 conv2d_385
23 batch_normalization_382
24 batch_normalization_385
25 activation_382
26 activation_385
27 average_pooling2d_36
28 conv2d_381
29 conv2d_383
30 conv2d_386
31 conv2d_387
32 batch_normalization_381
33 batch_normalization_383
34 batch_normalization_386
35 batch_normalization_387
36 activation_381
37 activation_383
38 activation_386
39 activation_387
40 mixed0
41 conv2d_391
42 batch_normalization_391
43 activation_391
44 conv2d_389
45 conv2d_392
46 batch_normalization_389
47 batch_normalization_392
48 activation_389
49 activation_392
50 average_pooling2d_37
51 conv2d_388
52

In [ ]:
model = tf.keras.Sequential([
  inception_v3,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(len(label_names))])

In [ ]:
model.compile(optimizer=tf.train.AdamOptimizer(learning_rate=0.01), 
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=["accuracy"])

In [ ]:
len(model.trainable_variables)

38

In [ ]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 5, 5, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 4098      
Total params: 21,806,882
Trainable params: 11,118,978
Non-trainable params: 10,687,904
_________________________________________________________________


In [ ]:
model.fit(x=X_train_preprocessed, y=y_train, batch_size=100, epochs=100, validation_split=0.1)

Epoch 1/100
13/54 [======>.......................] - ETA: 13:11 - loss: 5.2416 - acc: 0.4962